In [ ]:
# @title Environment Setup
import os, re, subprocess, sys
from pathlib import Path

def run(cmd):
    print('+', ' '.join(cmd))
    subprocess.run(cmd, check=True)

try:
    run(['apt-get', 'update'])
    apt = Path('apt.txt')
    if apt.exists():
        pkgs = [p.strip() for p in apt.read_text().splitlines() if p.strip()]
        if pkgs:
            run(['apt-get', 'install', '-y', *pkgs])
except Exception as e:
    print('Skipping apt-get install:', e)

gpu = subprocess.run(['bash','-lc','nvidia-smi'], capture_output=True, text=True)
has_gpu = gpu.returncode == 0
index = os.environ.get('PYTORCH_INDEX_URL')
if not index and has_gpu:
    m = re.search(r'CUDA Version: (\d+)\.(\d+)', gpu.stdout or gpu.stderr)
    if m:
        index = f"https://download.pytorch.org/whl/cu{m.group(1)}{m.group(2)}"
if not index:
    index = "https://download.pytorch.org/whl/cpu"
print('Using PyTorch index:', index)
try:
    run([sys.executable, '-m', 'pip', 'install', f'--index-url={index}', 'torch==2.3.0', 'torchvision==0.18.0', 'torchaudio==2.3.0'])
except subprocess.CalledProcessError:
    fallback = "https://download.pytorch.org/whl/cpu"
    print('PyTorch install failed, falling back to', fallback)
    run([sys.executable, '-m', 'pip', 'install', f'--index-url={fallback}', 'torch==2.3.0', 'torchvision==0.18.0', 'torchaudio==2.3.0'])

req = 'requirements-colab-gpu.txt' if has_gpu else 'requirements-colab-cpu.txt'
run([sys.executable, '-m', 'pip', 'install', '-r', req])

print('Python', sys.version)
subprocess.run(['ffmpeg', '-version'])
import torch, torchvision, torchaudio, soxr
print('torch', torch.__version__)
print('torchvision', torchvision.__version__)
print('torchaudio', torchaudio.__version__)
print('CUDA available:', torch.cuda.is_available())
print('Resampler backend:', 'soxr', getattr(soxr, '__version__', 'unknown'))
